In [62]:
# Import libraries

import numpy as np
from matplotlib import pyplot as plt


import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian

In [118]:
f: lambda x: x[0] ** 2 + x[1] ** 2 + x[2] ** 2
h1 :lambda x: ((x[0] ** 2) / 4) + ((x[1] ** 2) / 5) + ((x[2] ** 2) / 25) - 1
h2: lambda x: x[0] + x[1] - x[2]
x = Variable(torch.tensor([1.,1.,1.]), requires_grad=True)


We have $n=3$ and $m=2$ here, so the DOF of systme is $n-m = 1$ \
meanwe have only one decision variable variable and two state variable \
let say: \ 
$x_1 $ here is the decision variable \
$ x_2$  and  $x_3$ are the state variables \

In [81]:
# Calculating Reduce Gradient
def RJ(f,h1,h2,x):
  Jocobian = torch.zeros((3, 3))
  Jocobian[0] = jacobian(f, (x))
  Jocobian[1] = jacobian(h1, (x))
  Jocobian[2] = jacobian(h2, (x))

  dfdd = Jocobian[0,0]
  dfds = Jocobian[0,1:]
  dhds = Jocobian[1:,1:]
  dhdd = Jocobian[1:,0]

  DfDd =  dfdd -torch.matmul(torch.matmul(dfds,torch.pinverse(dhds)),dhdd)
  return  DfDd, dfdd, dfds, dhds, dhdd

In [115]:
def LM(x):
  Lambda = 1. 
  norm = torch.norm(torch.tensor([h1(x),h2(x)]))
  while norm >1e-06 :
      ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
      with torch.no_grad():
          x[1:] = x[1:]- torch.matmul(torch.matmul(torch.pinverse(torch.matmul(dhds.T,dhds) + Lambda* torch.eye(2)), dhds.T),torch.tensor([h1(x),h2(x)]))
      norm= torch.norm(torch.tensor([h1(x),h2(x)]))
      print(norm)
  return x

    

In [119]:
LM(x)

tensor(0.6597)
tensor(0.5301)
tensor(0.4678)
tensor(0.4130)
tensor(0.3578)
tensor(0.3035)
tensor(0.2522)
tensor(0.2057)
tensor(0.1652)
tensor(0.1308)
tensor(0.1025)
tensor(0.0796)
tensor(0.0614)
tensor(0.0471)
tensor(0.0360)
tensor(0.0274)
tensor(0.0208)
tensor(0.0158)
tensor(0.0119)
tensor(0.0090)
tensor(0.0068)
tensor(0.0052)
tensor(0.0039)
tensor(0.0029)
tensor(0.0022)
tensor(0.0017)
tensor(0.0013)
tensor(0.0009)
tensor(0.0007)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(9.8939e-05)
tensor(7.4536e-05)
tensor(5.6264e-05)
tensor(4.2270e-05)
tensor(3.1862e-05)
tensor(2.4113e-05)
tensor(1.8100e-05)
tensor(1.3707e-05)
tensor(1.0351e-05)
tensor(7.6934e-06)
tensor(5.9605e-06)
tensor(4.4544e-06)
tensor(3.3570e-06)
tensor(2.4899e-06)
tensor(1.9660e-06)
tensor(1.3328e-06)
tensor(1.0991e-06)
tensor(8.5963e-07)


tensor([1.0000, 1.5614, 2.5614], requires_grad=True)

In [127]:
def newX(x, alpha):
    updatedx = torch.zeros(3)
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    updatedx[0] = x[0] - alpha * ReduceJacobian

    updatedx[1:] = x[1:] + (alpha * (torch.matmul(torch.pinverse(dhds),dhdd))*dfdd)
    return updatedx


def lineSearch(x, t0=0.5, K=25):
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    alpha = 1
    i = 0
    func = f(newX(x, alpha))
    phi = f(x) - (t0 * alpha * (ReduceJacobian ** 2))
    
    while func > phi and i < K:
        alpha = 0.5 * alpha
        func = f(newX(x, alpha))
        phi = f(x)- (t0 * alpha * (ReduceJacobian ** 2))
        i += 1
    return alpha

In [122]:

# Part 4
# Initialization
def GRG(x):
    ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)
    e = torch.norm(ReduceJacobian)
    tol = 1e-3

    xSol = x.detach().numpy()
    fVal = [f(x).item()]
    alphaSol = [1]
    eVal = [e]

    k = 0
    while e > tol:
        alpha = lineSearch(x)

        ReduceJacobian, dfdd, dfds, dhds, dhdd= RJ(f,h1,h2,x)

        with torch.no_grad():
            x[0] = x[0] - alpha * ReduceJacobian
            x[1:] = x[1:] + (alpha * np.matmul(torch.pinverse(dhds) ,  dhdd) *  ReduceJacobian)

   
        x = LM(x)
        xSol = np.vstack((xSol, x.detach().numpy())) 

        e = torch.norm(ReduceJacobian)
        
    return xSol

In [128]:
x = torch.tensor([1., 1., 1.], dtype=torch.float64, requires_grad=True)
xSol= GRG(x)

RuntimeError: ignored